In [4]:
# Import from local package
import sys
sys.path.append('../model')
import torch
import numpy as np
from tqdm import tqdm as progress_bar
import numba

# Device configuration
device = torch.device("cpu")

In [5]:
import matplotlib.pyplot as plt

In [6]:
from models import SimpleCNN3Layer as Model
from collectdata import collect_data

ImportError: attempted relative import with no known parent package

In [ ]:
validation = collect_data(
    '/share/lazy/schreihf/PvFinder/Oct03_20K_val.npz',
    batch_size=1)

In [ ]:
name = '../model/output3/20180815_120000_3layer_30.pyt'
model = Model().to(device)
model.load_state_dict(torch.load(name))
model.eval()

## Old version

In [ ]:
# Check False Positive
def testNet2(model, test_loader):
    FP_list=[]
    N=0
    Times=0
    NO_Batch=0
    FP=0
    
    
    with torch.no_grad():
        for inputs, labels in progress_bar(test_loader):
            NO_Batch = NO_Batch +1
            
            #Forward pass
            outputs = model(inputs)
            
            
            prob=0
            Tprob=0
            j=0

            while j < 3900:
                for i in range (j,j+1):
                    if outputs[0][i].item()> 1e-2 and labels[0][i].item()==0:
                        for jj in range (i,i+10):
                            Tprob+=labels[0][jj].item()
                            prob+=outputs[0][jj].item()
                            if outputs[0][jj]>0.01:
                                Times=Times+1      
                        if Tprob<0.9 and prob>0.2 and Times>=3:
                            FP=FP+1
                            FP_list.append((NO_Batch, j))
                            j+=10
                        else:
                            j+=1
                    else:
                        j+=1
                            
    return FP

In [ ]:
FP = testNet2(model, validation)
print(FP)

## Modern Version

In [ ]:
@numba.jit(numba.int32(numba.float32[:],numba.float32[:]))
def look_for_missing(outputs, labels):
    prob = 0
    Tprob = 0
    j = 0
    FP = 0

    while j < 3900:
        
        for i in range (j, j+1):
            
            # If output value and not label value
            if outputs[i] > 1e-2 and labels[i] == 0:
                
                # Scan the next ten cells
                for jj in range(i, i+10):
                    Tprob += labels[jj]
                    prob += outputs[jj]
                    
                        
                # If label_prob is small and output_prob is large,
                # count a false positive and move on 10 cells
                if Tprob < 0.9 and prob > 0.2:
                    FP += 1
                    j += 10
                else:
                    j += 1
            else:
                j += 1
                
    return FP

In [ ]:
# Check False Positive
def testNet3(model, test_loader):
    FP_list=[]
    N=0
    NO_Batch=0
    FP=0
    
    
    with torch.no_grad():
        for inputs, labels in progress_bar(test_loader):
            NO_Batch = NO_Batch +1
            
            #Forward pass
            outputs = model(inputs)
            
            
            value = look_for_missing(outputs[0].numpy(), labels[0].numpy())
            FP += value
                            
    return FP

In [ ]:
FP3 = testNet3(model, validation)
print(FP3)

In [ ]:
i = 154
Y = validation.dataset.tensors[1][i]
with torch.no_grad():
    out = model(validation.dataset.tensors[0][i][None,:,:])

truth = Y.numpy()
predict = out.numpy().squeeze()

# Find the locations of values so that we can set limits
non_zero, = np.nonzero(np.round(truth + predict, 4))
    
plt.figure(figsize=(18,2))
plt.plot(-truth, label='Truth')
plt.plot(predict, label='Prediction')
plt.xlim(min(non_zero) - 20, max(non_zero) + 400)
plt.legend();

In [ ]:
@numba.jit(numba.float32[:](numba.float32[:], numba.float32), nopython=True)
def pv_locations(targets, threshold):
    state = False
    start = 0
    items = np.empty(150, np.float32)
    nitems = 0
    
    for i in range(len(targets)):
        if targets[i] >= threshold and not state:
            state = True
            start = i
        elif targets[i] < threshold and state:
            state = False
            items[nitems] = (i + start) / 2.
            nitems += 1
        # otherwise, keep going
    return items[:nitems]

In [ ]:
pv_locations(truth,1e-2)

In [ ]:
pv_locations(predict, 1e-2)

In [ ]:
%%timeit
pv_locations(predict, 1e-2)

In [ ]:
%%timeit
pv_locations(predict, 1e-2)

In [ ]:
@numba.jit(numba.types.UniTuple(numba.int32,2)(numba.float32[:], numba.float32[:], numba.float32), nopython=True)
def compare(a, b, diff):
    succeed = 0
    fail = 0
    
    # Check for closest value
    for item in a:
        mindiff = np.abs(b-item).min()
        if mindiff > diff:
            fail += 1
        else:
            succeed += 1
    
    return succeed, fail

In [ ]:
a = pv_locations(truth, 1e-2)
b = pv_locations(predict, 1e-2)

print(*compare(a, b, 3.))
print(*compare(b, a, 3.))

In [ ]:
@numba.jit(numba.types.UniTuple(numba.int32,3)(numba.float32[:], numba.float32[:], numba.float32, numba.float32))
def efficency(truth, predict, threshold, difference):
    """
    Compute three value: The number of succeses (S), the number of missed true values (MT), and the number of missed false values (FP).
    
    Accepts:
      * truth: Numpy array of truth values
      * predict: Numpy array of predictions
      * threshold: The threshold for considering an "on" value
      * difference: The maximum difference to count a success, in bin widths (successes and failures are to the nearest half bin, currently)
    
    Returns: S, MT, FP
    
    A future advancement of this algorithm would be to compute the weighted mean, and use that.
    Also, this will currently be triggered by small fluctionations in the input array. It should have a minium total integrated value
    required to "turn it on".
    """
    
    a = pv_locations(truth, threshold)
    b = pv_locations(predict, threshold)

    S, MT = compare(a, b, difference)
    Sp, FP = compare(b, a, difference)
    
    assert S == Sp
    
    return S, MT, FP

In [ ]:
%%timeit
efficency(truth, predict, 1e-2, 20.)